Notebook to query 

1) Amazon Bedrock LLM / Foundation Model and then 

2) enhance with Retreival Augmented Generation (RAG) from pre-prepared knowledge base from LDFE job status reports

References:

https://youtu.be/BXgaK8PPZAE

### Claude (Anthropic Model) - after use case approval

In [2]:
import boto3
from langchain.llms.bedrock import Bedrock

#### Step 1: using Anthropic Claude 2.1 for Amazon Bedrock to propose a list of questions to ask in relation to data anomalies for forecasting time series data

In [3]:
bedrock_client = boto3.client("bedrock-runtime", region_name = 'us-east-1')

# Start with the query
# *check* possibly mention labour demand forecasting 
prompt = "For forecasting time series data, what are the key questions to ask in relation to data anomalies ?"

claude_llm = Bedrock(
    model_id="anthropic.claude-v2:1",
    model_kwargs={"temperature": 0, "top_k": 10, "max_tokens_to_sample": 3000},
    client=bedrock_client,
)

# Provide the prompt to the LLM to generate an answer to the query without any additional context provided
response = claude_llm(prompt)
questions = [
    item.split(".")[1].strip() for item in response.strip().split("\n\n")[1:-1]
]

/Users/barry.walsh/rotaready/rr_repos/ML-LabourDemandForecasting/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [4]:
questions

["It's important to identify and potentially remove or adjust any anomalous data points",
 'Consider whether and how to fill in or otherwise adjust for any missing values',
 '',
 'Plotting the data over time can help spot changes in seasonality',
 'This may require using autoregressive forecasting models',
 '']

#### Step 2: (fully managed RAG) - use native Knowledge Bases (based on s3 bucket with LDFE training and inference job statuses) for Amazon Bedrock RetrieveAndGenerate API to obtain answers directly

In [24]:
reg_id = "us-east-1" # for now only this supported
kb_id = "O3SYXISL4R"

In [25]:
bedrock_agent_client = boto3.client("bedrock-agent-runtime", region_name = reg_id)

def retrieveAndGenerate(
    input: str,
    kbId: str,
    region: str = reg_id,
    sessionId: str = None,
    model_id: str = "anthropic.claude-v2", # changed from "anthropic.claude-v2:1",
):
    model_arn = f"arn:aws:bedrock:{region}::foundation-model/{model_id}"

    if sessionId:
        return bedrock_agent_client.retrieve_and_generate(
            input={"text": input},
            retrieveAndGenerateConfiguration={
                "type": "KNOWLEDGE_BASE",
                "knowledgeBaseConfiguration": {
                    "knowledgeBaseId": kbId,
                    "modelArn": model_arn,
                },
            },
            sessionId=sessionId,
        )

    else:
        return bedrock_agent_client.retrieve_and_generate(
            input={"text": input},
            retrieveAndGenerateConfiguration={
                "type": "KNOWLEDGE_BASE",
                "knowledgeBaseConfiguration": {
                    "knowledgeBaseId": kbId,
                    "modelArn": model_arn,
                },
            },
        )

#target_qu = "What are the key anomalies we are seeing in the model training process ?"
target_qu = "What are the main errors we are seeing in the inference process ?"


response = retrieveAndGenerate(
    target_qu, kb_id
)

generated_text = response["output"]["text"]

In [26]:
# get custom responses to the question posed in the last cell
generated_text

'The main errors we are seeing in the inference process are insufficient rota data and no model or proxy model available in S3.'

*THE ABOVE RESPONSE SOUNDS GOOD AND IS ACTUALLY CORRECT ON CHECKING THE SOURCE DATA AND LOOKING AT THE "NOTE" FIELD *

In [27]:
# cite evidence for the above response
response["citations"]


[{'generatedResponsePart': {'textResponsePart': {'span': {'end': 125,
     'start': 0},
    'text': 'The main errors we are seeing in the inference process are insufficient rota data and no model or proxy model available in S3.'}},
  'retrievedReferences': [{'content': {'text': '31424,13/03/2024 03:08,brewdog,haw,inference,success,"[\'\', \'\']", 31425,13/03/2024 03:08,brewdog,brs,inference,success,"[\'\', \'\']", 31426,13/03/2024 03:08,brewdog,heo,inference,fail,[\'insufficient rota data - no model or proxy model in S3 available\'],"brewdog-heo has insufficient data, no model" 31427,13/03/2024 03:08,brewdog,wac,inference,success,"[\'\', \'\']", 31428,13/03/2024 03:09,brewdog,can,inference,success,"[\'\', \'\']", 31429,13/03/2024 03:09,brewdog,mii,inference,success,"[\'\', \'\']", 31430,13/03/2024 03:09,brewdog,car,inference,success,"[\'\', \'\']", 31431,13/03/2024 03:09,brewdog,nic,inference,success,"[\'\', \'\']", 31432,13/03/2024 03:09,brewdog,mil,inference,success,"[\'\', \'\']", 3

In [28]:
session_id = response.get('sessionId')

In [29]:
session_id

'23bd3704-8be1-4cab-ad91-aaaeca7d9ad2'

In [ ]:
# follow up question with no context

#followUpQu = "how many jobs failed with the train_size None error" # training
followUpQu = "is the same error the main cause of runs failing this morning ?" # inference

# more detailed
#followUpQu = "can you tell me approximately how many training jobs failed within the last month with the train_size=None error and what percentage this is of all the training jobs in that period"

retrieveAndGenerate(followUpQu, kb_id)["output"]["text"]

* THE ABOVE RESPONSE IS POOR ON A FEW LEVELS - IT SHOULD START "No...." as it goes on to highlight a different error than the original one. Also on checking the source of the KB, the answer is incorrect. It also assumes the user is still referring to inference runs, rather than both training and inference

In [31]:
# get context-specific reponse (i.e. considering original response)
# *check* throws an error atm about use of wildcards in sess_id ?
sess_id = response["sessionId"]
retrieveAndGenerate(followUpQu, kb_id, reg_id, sess_id)["output"]["text"]

'Yes, insufficient rota data seems to be the main cause of inference run failures this morning.'

* THIS RESPONSE RETAINS CONTEXT AND WHILE A BIT SPARSE, FOLLOWS ON FROM THE ORIGINAL QUESTION

#### Step 3: RAG customisation - uses the Retrieve API to fetch the relevant chunks based on your query and pass it to any LLM provided by Amazon Bedrock



In [32]:
def retrieve(query: str, kbId: str, numberOfResults: int = 5):
    return bedrock_agent_client.retrieve(
        retrievalQuery={"text": query},
        knowledgeBaseId=kbId,
        retrievalConfiguration={
            "vectorSearchConfiguration": {"numberOfResults": numberOfResults}
        },
    )

##### 3.1 get the correponding KB context for a query

In [33]:
query = "How many failed inference jobs today mentioned busyness ?"
response = retrieve(query, kb_id, 3)
retrievalResults = response["retrievalResults"]

In [34]:
retrievalResults

[{'content': {'text': ', \'\', \'\']", 32433,19/03/2024 09:02,nq64,car,inference,success,"[\'\', \'inconsistent busyness data - check\', \'\']", 32434,19/03/2024 09:02,brewdog,che,inference,success,"[\'\', \'\', \'\']", 32435,19/03/2024 09:02,warnerleisure,ahs,inference,success,"[\'\', \'unable to capture google places data\', \'\']", 32436,19/03/2024 09:02,spaceandthyme,kit,inference,success,"[\'\', \'unable to capture google places data\', \'\']", 32437,19/03/2024 09:02,brewdog,cbg,inference,success,"[\'\', \'\', \'\']", 32438,19/03/2024 09:02,brewdog,bou,inference,success,"[\'\', \'\', \'\']", 32439,19/03/2024 09:02,namco,nwc,inference,success,"[\'\', \'\', \'\']", 32440,19/03/2024 09:02,bonnieandwild,kps,inference,success,"[\'\', \'\', \'\']", 32441,19/03/2024'},
  'location': {'s3Location': {'uri': 's3://ldfe-genai/debug/inference/job_status.csv'},
   'type': 'S3'},
  'metadata': {'x-amz-bedrock-kb-source-uri': 's3://ldfe-genai/debug/inference/job_status.csv'},
  'score': 0.546943

##### 3.2 extract the context for the LLM / FM prompt template

In [35]:
def get_contexts(retrievalResults):
    contexts = []
    for retrievedResult in retrievalResults:
        contexts.append(retrievedResult["content"]["text"])
    return " ".join(contexts)

contexts = get_contexts(retrievalResults)
contexts

', \'\', \'\']", 32433,19/03/2024 09:02,nq64,car,inference,success,"[\'\', \'inconsistent busyness data - check\', \'\']", 32434,19/03/2024 09:02,brewdog,che,inference,success,"[\'\', \'\', \'\']", 32435,19/03/2024 09:02,warnerleisure,ahs,inference,success,"[\'\', \'unable to capture google places data\', \'\']", 32436,19/03/2024 09:02,spaceandthyme,kit,inference,success,"[\'\', \'unable to capture google places data\', \'\']", 32437,19/03/2024 09:02,brewdog,cbg,inference,success,"[\'\', \'\', \'\']", 32438,19/03/2024 09:02,brewdog,bou,inference,success,"[\'\', \'\', \'\']", 32439,19/03/2024 09:02,namco,nwc,inference,success,"[\'\', \'\', \'\']", 32440,19/03/2024 09:02,bonnieandwild,kps,inference,success,"[\'\', \'\', \'\']", 32441,19/03/2024 [\'\', \'inconsistent busyness data - check\', \'\']", 42421,20/04/2024 03:05,warnerleisure,cog,inference,fail,[\'insufficient rota data - no model or proxy model in S3 available\'],"warnerleisure-cog has insufficient data, no model" 42422,20/04/2

##### 3.3 set up the LLM / FM in-context question-answering prompt template, then generate the final answer

In [36]:
from langchain.prompts import PromptTemplate

# # question NOT query_str for langchain step4 - see https://stackoverflow.com/questions/77839844/langchain-retrievalqa-missing-some-input-keys
PROMPT_TEMPLATE = """
Human: You are an AI system working on forecast anomalies, and provides answers to questions \
by using fact based and statistical information when possible.
Use the following pieces of information to provide a concise answer to the question enclosed in <question> tags.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

<context>
{context}
</context>

<question>
{question}
</question>

The response should be specific and use statistics or numbers when possible.

Assistant:"""

claude_prompt = PromptTemplate(
    template=PROMPT_TEMPLATE, input_variables=["context", "question"] # question NOT query_str for langchain step4 - see https://stackoverflow.com/questions/77839844/langchain-retrievalqa-missing-some-input-keys
)

prompt = claude_prompt.format(context=contexts, question=query) # question NOT query_str for langchain step4 - see https://stackoverflow.com/questions/77839844/langchain-retrievalqa-missing-some-input-keys
response = claude_llm(prompt)

In [37]:
response

' Unfortunately, I do not have enough information to definitively state how many failed inference jobs today mentioned busyness. The context provided includes some failed inference jobs with messages about inconsistent busyness data, but does not specify the date these jobs ran or the total number of failed inference jobs. Without additional details on the total failed jobs and dates, I cannot provide a statistic on how many mentioned busyness.'

* THIS APPROACH IS MORE SOPHISTICATED AS IT USES THE CONTEXT SPECIFIC KNOWLEDGE BASE THEN, CHUNKS UP THE REVELANT VECTORS RELATING TO THE QUESTION, THEN HANDS-OFF TO THE LLM (ANTHROPIC) TO PROVIDE A MORE INFORMED / DETAILED RESPONSE

#### Step 4: Amazon Bedrock LangChain integration - basically step 3 *check* integrated with the LangChain framework designed to simplify the creation of applications using large language models. So used to create an end-to-end customized Q&A application.



In [38]:
# 4.1 set up the LangChain retriever and specificy number of results to return

from langchain.retrievers.bedrock import AmazonKnowledgeBasesRetriever

retriever = AmazonKnowledgeBasesRetriever(
    knowledge_base_id=kb_id,
    region_name="us-east-1",
    retrieval_config={"vectorSearchConfiguration": {"numberOfResults": 4}},
)

In [39]:
# 4.2 set up the LangChain RetrievalQA and generate answers from the knowledge base:

from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=claude_llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": claude_prompt}, # same prompt template as in step 3
)

#[qa(q)["result"] for q in questions]

[qa(query)["result"]]

/Users/barry.walsh/rotaready/rr_repos/ML-LabourDemandForecasting/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


[" Unfortunately, there is no information provided in the context about failed inference jobs mentioning busyness today. The context shows inference job results from various dates, but does not give a count of failed jobs mentioning busyness on any specific date. So I don't have enough information to provide a numerical answer about failed inference jobs mentioning busyness today."]

* THIS APPROACH IS I THINK THE PREFERRED APPROACH AS USES LANGCHAIN SIMPLIFIED INTEGRATION FOR GENAI APPS

In [ ]:
questions

#### Amazon Titan Express

Reference:

https://aws.amazon.com/blogs/machine-learning/getting-started-with-amazon-titan-text-embeddings/

https://docs.aws.amazon.com/bedrock/latest/userguide/titan-models.html 

In [ ]:
# NB text EMBEDDINGS refer to the NLP technique that converts textual data into numerical vectors that can be processed by ML algos and LLMs.
# They are more suited for use cases where SEMANTICS are important (capturing meaning of singular words or phrases)

# *check* differences between Amazon Titan text and embeddings models. Embeddings seems to be cheaper, possibly due to the storage of the numercal/vector representation of text, as opposed to the untransformed text format, but link below suggests (vanilla) text model is high performance.

In [ ]:
# print out avaialble Amazon Titan LLM models

import boto3
import json
 
#Create the connection to Bedrock
bedrock = boto3.client(
    service_name='bedrock',
    region_name='us-east-1', 
    
)
 
bedrock_runtime = boto3.client(
    service_name='bedrock-runtime',
    region_name='us-east-1', 
    
)
 
# Let's see all available Amazon Models
available_models = bedrock.list_foundation_models()
 
for model in available_models['modelSummaries']:
  if 'amazon' in model['modelId']:
    print(model)
 


In [ ]:
# Prompt Engineering example - have Amazon Titan write a poem about apples

# Define prompt and model parameters
prompt_data = """Write me a poem about apples"""
 
body = json.dumps({
    "inputText": prompt_data,
})
 
model_id = 'amazon.titan-text-express-v1' #'amazon.titan-embed-text-v1' #look for embeddings in the modelID
accept = 'application/json' 
content_type = 'application/json'
 
# Invoke model 
response = bedrock_runtime.invoke_model(
    body=body, 
    modelId=model_id, 
    accept=accept, 
    contentType=content_type
)
 
# Print response
response_body = json.loads(response['body'].read())

print(response_body)

# embedding = response_body.get('embedding')
 
# #Print the Embedding
 
# print(embedding)

In [ ]:
results = response_body.get('results')
results[0]['outputText']

#### Scrap

Code sample taken from:

https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-titan-text.html

In [ ]:
# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: Apache-2.0
"""
Shows how to create a list of action items from a meeting transcript
with the Amazon &titan-text-express; model (on demand).
"""
import json
import logging
import boto3

from botocore.exceptions import ClientError


class ImageError(Exception):
    "Custom exception for errors returned by Amazon &titan-text-express; model"

    def __init__(self, message):
        self.message = message


logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)


def generate_text(model_id, body):
    """
    Generate text using Amazon &titan-text-express; model on demand.
    Args:
        model_id (str): The model ID to use.
        body (str) : The request body to use.
    Returns:
        response (json): The response from the model.
    """

    logger.info(
        "Generating text with Amazon &titan-text-express; model %s", model_id)

    bedrock = boto3.client(service_name='bedrock-runtime')

    accept = "application/json"
    content_type = "application/json"

    response = bedrock.invoke_model(
        body=body, modelId=model_id, accept=accept, contentType=content_type
    )
    response_body = json.loads(response.get("body").read())

    finish_reason = response_body.get("error")

    if finish_reason is not None:
        raise ImageError(f"Text generation error. Error is {finish_reason}")

    logger.info(
        "Successfully generated text with Amazon &titan-text-express; model %s", model_id)

    return response_body


def main():
    """
    Entrypoint for Amazon &titan-text-express; example.
    """
    try:
        logging.basicConfig(level=logging.INFO,
                            format="%(levelname)s: %(message)s")

        model_id = 'amazon.titan-text-express-v1'

        prompt = """Meeting transcript: Miguel: Hi Brant, I want to discuss the workstream  
            for our new product launch Brant: Sure Miguel, is there anything in particular you want
            to discuss? Miguel: Yes, I want to talk about how users enter into the product.
            Brant: Ok, in that case let me add in Namita. Namita: Hey everyone 
            Brant: Hi Namita, Miguel wants to discuss how users enter into the product.
            Miguel: its too complicated and we should remove friction.  
            for example, why do I need to fill out additional forms?  
            I also find it difficult to find where to access the product
            when I first land on the landing page. Brant: I would also add that
            I think there are too many steps. Namita: Ok, I can work on the
            landing page to make the product more discoverable but brant
            can you work on the additonal forms? Brant: Yes but I would need 
            to work with James from another team as he needs to unblock the sign up workflow.
            Miguel can you document any other concerns so that I can discuss with James only once?
            Miguel: Sure.
            From the meeting transcript above, Create a list of action items for each person. """

        body = json.dumps({
            "inputText": prompt,
            "textGenerationConfig": {
                "maxTokenCount": 4096,
                "stopSequences": [],
                "temperature": 0,
                "topP": 1
            }
        })

        response_body = generate_text(model_id, body)
        print(f"Input token count: {response_body['inputTextTokenCount']}")

        for result in response_body['results']:
            print(f"Token count: {result['tokenCount']}")
            print(f"Output text: {result['outputText']}")
            print(f"Completion reason: {result['completionReason']}")

    except ClientError as err:
        message = err.response["Error"]["Message"]
        logger.error("A client error occurred: %s", message)
        print("A client error occured: " +
              format(message))
    except ImageError as err:
        logger.error(err.message)
        print(err.message)

    else:
        print(
            f"Finished generating text with the Amazon &titan-text-express; model {model_id}.")


if __name__ == "__main__":
    main()

In [ ]:
import boto3
from langchain.llms.bedrock import Bedrock

#bedrock_client = boto3.client("bedrock-runtime")
bedrock_client = boto3.client(service_name='bedrock',
                              region_name='us-east-1',
                              endpoint_url='https://bedrock.us-east-1.amazonaws.com',)

# Start with the query
# *check* possibly mention labour demand forecasting 
prompt = "For forecasting time series data, what are the key questions to ask in relation to data anomalies ?"

titan_llm = Bedrock(
    model_id="amazon.titan-text-express-v1", # modelId on AWS Console > Bedrock > Providers > https://us-east-1.console.aws.amazon.com/bedrock/home?region=us-east-1#/providers?model=amazon.titan-text-express-v1
    model_kwargs={"temperature": 0, "top_k": 10, "max_tokens_to_sample": 3000},
    client=bedrock_client,
)

# Provide the prompt to the LLM to generate an answer to the query without any additional context provided
response = titan_llm(prompt)
questions = [
    item.split(".")[1].strip() for item in response.strip().split("\n\n")[1:-1]
]

In [ ]:
def get_foundation_model(model_identifier):
    """
    Get details about an Amazon Bedrock foundation model.

    :return: The foundation model's details.
    """

    try:
        return bedrock_client.get_foundation_model(
            modelIdentifier=model_identifier
        )["modelDetails"]
    except ClientError:
        logger.error(
            f"Couldn't get foundation models details for {model_identifier}"
        )
        raise

In [ ]:
get_foundation_model('titan_llm')

In [ ]:
from langchain.llms import Bedrock

llm = Bedrock(
    credentials_profile_name="default",
    model_id="amazon.titan-text-express-v1"
)

from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

conversation = ConversationChain(
    llm=llm, verbose=True, memory=ConversationBufferMemory()
)

result = conversation.predict(input="What is the most exciting question you can ask an LLM?")
print(result)

In [ ]:
# Reference:
# https://aws.amazon.com/blogs/machine-learning/getting-started-with-amazon-titan-text-embeddings/

import boto3
import json
 
#Create the connection to Bedrock
bedrock = boto3.client(
    service_name='bedrock',
    region_name='us-east-1', 
    
)
 
bedrock_runtime = boto3.client(
    service_name='bedrock-runtime',
    region_name='us-east-1', 
    
)
 
# Let's see all available Amazon Models
available_models = bedrock.list_foundation_models()
 
for model in available_models['modelSummaries']:
  if 'amazon' in model['modelId']:
    print(model)
 
# Define prompt and model parameters
prompt_data = """Write me a poem about apples"""
 
body = json.dumps({
    "inputText": prompt_data,
})
 
model_id = 'amazon.titan-text-express-v1' #'amazon.titan-embed-text-v1' #look for embeddings in the modelID
accept = 'application/json' 
content_type = 'application/json'
 
# Invoke model 
response = bedrock_runtime.invoke_model(
    body=body, 
    modelId=model_id, 
    accept=accept, 
    contentType=content_type
)
 
# Print response
response_body = json.loads(response['body'].read())
embedding = response_body.get('embedding')
 
#Print the Embedding
 
print(embedding)

In [ ]:
print(response_body)